## Yolo V3 with Tensorflow

In [2]:
from net.yolo_top import yolov3
from data.data_pipeline import data_pipeline
from net.config import cfg
import numpy as np
import time
import tensorflow as tf
import os
% load_ext autoreload
% autoreload 2

In [3]:
MOVING_AVERAGE_DECAY = 0.9999     # The decay to use for the moving average.
NUM_EPOCHS_PER_DECAY = 350.0      # Epochs after which learning rate decays.
LEARNING_RATE_DECAY_FACTOR = 0.1  # Learning rate decay factor.
INITIAL_LEARNING_RATE = 0.1       # Initial learning rate.

In [4]:
def tower_loss(scope, imgs, true_boxes, istraining):
    """Calculate the total loss on a single tower running the CIFAR model.
    Args:
        scope: unique prefix string identifying the CIFAR tower, e.g. 'tower_0'
        images: Images. 4D tensor of shape [batch_size, height, width, 3].
        labels: Labels. 1D tensor of shape [batch_size].
    Returns:
        Tensor of shape [] containing the total loss for a batch of data
    """

    # Build inference Graph.
    # Build the portion of the Graph calculating the losses. Note that we will
    # assemble the total_loss using a custom function below.
    model = yolov3(imgs, true_boxes, istraining)
    _ = model.compute_loss()

    # Assemble all of the losses for the current tower only.
    losses = tf.get_collection('losses', scope)
        
    # Calculate the total loss for the current tower.
    total_loss = tf.add_n(losses, name='total_loss')
    
    return total_loss

def average_gradients(tower_grads):
    """Calculate the average gradient for each shared variable across all towers.
    Note that this function provides a synchronization point across all towers.
    Args:
        tower_grads: List of lists of (gradient, variable) tuples. The outer list
        is over individual gradients. The inner list is over the gradient
        calculation for each tower.
    Returns:
        List of pairs of (gradient, variable) where the gradient has been averaged
        across all towers.
    """
    average_grads = []

    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:            
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(axis=0, values=grads)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [5]:
def train(num_gpus):

    with tf.Graph().as_default(), tf.device('/cpu:0'):
        # Create a variable to count the number of train() calls. This equals the
        # number of batches processed * FLAGS.num_gpus.
        global_step = tf.Variable(0, trainable=False)
        
        decay_steps = int(100 * 100)

        #lr = tf.train.exponential_decay(INITIAL_LEARNING_RATE, 
        #                                 global_step,
        #                                 decay_steps,
        #                                 LEARNING_RATE_DECAY_FACTOR,
        #                                 staircase=True)

        lr = tf.train.piecewise_constant(global_step, 
                                         cfg.train.lr_steps,
                                         cfg.train.lr_scales)
        
        # Create an optimizer that performs gradient descent.
        
#        opt = tf.train.GradientDescentOptimizer(lr)
        
        opt = tf.train.AdamOptimizer(learning_rate = lr)
        
        tower_grads = []
        
        imgs, true_boxes = data_pipeline(
            [cfg.path.train_data_path, cfg.path.train_data_path], cfg.batch_size)
        
        print(imgs.shape)
        print(true_boxes.shape)
        
        imgs = tf.reshape(imgs, 
                          [cfg.batch_size, imgs.shape[1], imgs.shape[2], imgs.shape[3]])
        true_boxes = tf.reshape(true_boxes, 
                                [cfg.batch_size, true_boxes.shape[1], true_boxes.shape[2]])
        
        batch_queue = tf.contrib.slim.prefetch_queue.prefetch_queue(
            [imgs, true_boxes], capacity=2 * num_gpus)
        
        with tf.variable_scope(tf.get_variable_scope()):
            for i in range(num_gpus):
                with tf.device('/gpu:%d' % i):
                    with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                        istraining = tf.constant(True, tf.bool)
                        
                        imgs_batch, true_boxes_batch = batch_queue.dequeue()
                        # Calculate the loss for one tower of the CIFAR model. This function
                        # constructs the entire CIFAR model but shares the variables across
                        # all towers.
                        
                        loss = tower_loss(scope, 
                                          imgs_batch, 
                                          true_boxes_batch,
                                          istraining)

                        # Reuse variables for the next tower.
                        tf.get_variable_scope().reuse_variables()

                        # Retain the summaries from the final tower.
                        summaries = tf.get_collection(tf.GraphKeys.SUMMARIES, scope)

                        # Calculate the gradients for the batch of data on this CIFAR tower.
                        grads = opt.compute_gradients(loss)
                        
                        print('ops________________________________________________')
                        for i in grads:
                            print(i[0])
                        print ('variables________________________________________________')
                        for i in grads:
                            print (i[1])

                        # Keep track of the gradients across all towers.
                        tower_grads.append(grads)

        # We must calculate the mean of each gradient. Note that this is the
        # synchronization point across all towers.

        grads = average_gradients(tower_grads)

        print('average ops________________________________________________')
        for i in grads:
            print(i[0])
        print ('average variables________________________________________________')
        for i in grads:
            print (i[1])

        # Add a summary to track the learning rate.
        summaries.append(tf.summary.scalar('learning_rate', lr))

        # Add histograms for gradients.
        for grad, var in grads:
            if grad is not None:
                summaries.append(tf.summary.histogram(var.op.name + '/gradients', grad))

        # Apply the gradients to adjust the shared variables.
        apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)
        
        #update_op = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        #vars_det = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Head")
        
        #with tf.control_dependencies(update_op):
        #    apply_gradient_op = opt.minimize(loss, 
        #                                     global_step = global_step, 
        #                                     var_list = vars_det)
        
        # Add histograms for trainable variables.
        for var in tf.trainable_variables():
            summaries.append(tf.summary.histogram(var.op.name, var))

        # Group all updates to into a single train op.
        train_op = apply_gradient_op

        # Create a saver.
        saver = tf.train.Saver(tf.global_variables())

        # Build the summary operation from the last tower summaries.
        summary_op = tf.summary.merge(summaries)

        # Build an initialization operation to run below.
        init = tf.global_variables_initializer()

        # Start running operations on the Graph. allow_soft_placement must be set to
        # True to build towers on GPU, as some of the ops do not have GPU
        # implementations.
        sess = tf.Session(config=tf.ConfigProto(
            allow_soft_placement=True,
            log_device_placement=False))
        sess.run(init)
        
        # Start the queue runners.
        tf.train.start_queue_runners(sess=sess)

        for step in range(10000):
            
            start_time = time.time()
            _, loss_value = sess.run([train_op, loss])
            duration = time.time() - start_time

            print(duration)
            if (step % 10 == 0):
                print(loss_value)
                pass

        print("Complete!!")
        sess.close()
            
            

In [ ]:
train(1)

(?, 608, 608, 3)
(?, 30, 5)
build start
Tensor("tower_0/Feature_Extractor/conv_0/LeakyRelu/Maximum:0", shape=(6, 608, 608, 32), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/Feature_Extractor/conv_1/LeakyRelu/Maximum:0", shape=(6, 304, 304, 64), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/Feature_Extractor/conv_2/LeakyRelu/Maximum:0", shape=(6, 304, 304, 32), dtype=float32, device=/device:GPU:0)
build done
ops________________________________________________
Tensor("tower_0/gradients/tower_0/Feature_Extractor/conv_0/Conv2D_grad/tuple/control_dependency_1:0", shape=(3, 3, 3, 32), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/gradients/tower_0/Feature_Extractor/conv_0/batchnorm/sub_grad/tuple/control_dependency:0", shape=(32,), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/gradients/tower_0/Feature_Extractor/conv_0/batchnorm/mul_grad/tuple/control_dependency_1:0", shape=(32,), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/gradients/tower_0/Feature_Extr

average ops________________________________________________
Tensor("Mean:0", shape=(3, 3, 3, 32), dtype=float32, device=/device:CPU:0)
Tensor("Mean_1:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_2:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_3:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_4:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_5:0", shape=(3, 3, 32, 64), dtype=float32, device=/device:CPU:0)
Tensor("Mean_6:0", shape=(64,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_7:0", shape=(64,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_8:0", shape=(64,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_9:0", shape=(64,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_10:0", shape=(1, 1, 64, 32), dtype=float32, device=/device:CPU:0)
Tensor("Mean_11:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_12:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_13:

12.796285390853882
41126.5
0.5404543876647949
0.554692268371582
0.5603687763214111
0.5237557888031006
0.5533688068389893
0.5711474418640137
0.5515561103820801
0.5479581356048584
0.5432703495025635
0.573617696762085
16825.5
0.5331432819366455
0.5607531070709229
0.5498602390289307
0.5542728900909424
0.5244612693786621
0.5549213886260986
0.5545661449432373
0.552513837814331
0.5264081954956055
0.5617923736572266
7844.9
0.550370454788208
0.5367341041564941
0.5560486316680908
0.5708832740783691
0.5642518997192383
0.5189492702484131
0.5605120658874512
0.5505506992340088
0.5517256259918213
0.5303213596343994
4593.11
0.566650390625
0.5548059940338135
0.5220351219177246
0.548896312713623
0.5689127445220947
0.5521905422210693
0.523456335067749
0.5574324131011963
0.5477931499481201
0.5455596446990967
3665.84
0.549689769744873
0.5564179420471191
0.5522840023040771
0.5274324417114258
0.5499773025512695
0.5629043579101562
0.5520267486572266
0.5239455699920654
0.549950122833252
0.5595831871032715
2745

0.5468580722808838
0.5519638061523438
0.5592279434204102
0.5278198719024658
913.131
0.5597124099731445
0.5532855987548828
0.5649499893188477
0.5206594467163086
0.5624308586120605
0.5541503429412842
0.5453968048095703
0.5441012382507324
0.5566997528076172
0.5742490291595459
820.279
0.5172343254089355
0.567014217376709
0.5532705783843994
0.5450756549835205
0.5213031768798828
0.5538947582244873
0.5463626384735107
0.5431003570556641
0.5364847183227539
0.5556185245513916
581.922
0.5534346103668213
0.528479814529419
0.5524075031280518
0.566673755645752
0.5465729236602783
0.5166254043579102
0.5504884719848633
0.553598165512085
0.5423362255096436
0.5325601100921631
542.606
0.5586686134338379
0.5586471557617188
0.5240743160247803
0.5499823093414307
0.5555510520935059
0.5567536354064941
0.5182511806488037
0.5516970157623291
0.5627477169036865
0.5384387969970703
529.326
0.5526494979858398
0.5541069507598877
0.56467604637146
0.517258882522583
0.5502200126647949
0.5565383434295654
0.550458192825317